In [1]:
%load_ext autoreload
%autoreload 2

import typing
from typing import List, Iterable
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import rdkit
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles

from utils import *

import os, glob, gc

import gpytorch
import torch

from gauche.kernels.fingerprint_kernels.tanimoto_kernel import TanimotoKernel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

SEED = 0
P_INIT = 0.10
P_ITER = 0.05
N_ITER = 5
P_EXPLORE = 0.2

Loaded 97 descriptor functions
cuda:0


In [2]:
def cuda_mem():
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    print((t-a)/1024/1024/1024)
cuda_mem()

7.99560546875


In [3]:
with open("dataset/596/data.pkl","rb") as file:
    data = pickle.load(file)

fp_ls = data["fp_ls"][:20000]
y = np.array(data["activity_ls"], dtype=float)[:20000]

# initial picks using RDkit
init_idxs = random_pick(fp_ls, int(P_INIT*len(fp_ls)))

In [4]:
idx_to_nodes = data['idx_to_nodes']
node_to_hierarchy = data['node_to_hierarchy']

In [5]:
def hit_rate(y: np.array):
    return sum(y) / len(y)
def recovery_rate(y: np.array, idxs: Iterable):
    return sum(y[idxs]) / sum(y)
    
def iHTS(learner, y, init_idxs, n_iter=N_ITER, p_iter=P_ITER):
    #
    results = []
    
    # initialize idxs
    idxs = Indice(len(y))
    
    # observe initial set
    idxs.add(init_idxs)
    learner.observe(init_idxs, y[init_idxs])
    results.append(idxs.sampled)
    
    print(f"0 | {P_INIT:.2f}: {recovery_rate(y, idxs.sampled):.4f}")
    
    # iteratively sample the library
    for it in range(n_iter):
        
        sample_idxs = learner.select(idxs.unsampled, int(p_iter * len(y)))
        learner.observe(sample_idxs, y[sample_idxs])
        idxs.add(sample_idxs)

        results.append(idxs.sampled)
        
        print(f"{it+1} | {P_INIT+P_ITER*(it+1):.2f}: {recovery_rate(y, idxs.sampled):.4f}")

    results.append(idxs.unsampled)
    print("\n")
    return results

In [6]:
# def common_hierarchy(i, j, idx_to_nodes, node_to_hierarchy):
#     common_parents = list(set(idx_to_nodes[i]) & set(idx_to_nodes[i]))
#     if not common_parents:
#         return 0
#     else:
#         common_lowest_parents = common_parents[0]
#         return node_to_hierarchy[common_lowest_parents]

class ScaffoldTreeKernel(gpytorch.kernels.Kernel):
    is_stationary = True

    def set_tree_param(self, weights=[0,0,0.1,0.2,0.3,0.4]):
        self.weights = torch.tensor(weights, dtype=torch.float16).cuda()
    
    # this is the kernel function
    def forward(self, x1, x2, **params):
        size = x1.shape[0]
        return torch.stack([(((i==x2)&(i!=0)) * self.weights).sum(dim=1) for i in x1]).cuda()

In [7]:
scaffoldkernel = ScaffoldTreeKernel()
scaffoldkernel.set_tree_param()

In [8]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import UnwhitenedVariationalStrategy
from gauche.kernels.fingerprint_kernels.tanimoto_kernel import TanimotoKernel


class GPClassificationModel(ApproximateGP):
    def __init__(self, train_x):
        variational_distribution = CholeskyVariationalDistribution(train_x.size(0))
        variational_strategy = UnwhitenedVariationalStrategy(
            self, train_x, variational_distribution, learn_inducing_locations=False, jitter_val=1e-6
        )
        super(GPClassificationModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(scaffoldkernel)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred

In [11]:
class GP_learner(Learner):
    """Thompson sampling learner based on gaussian process"""
    
    def __init__(self, data):
        SIZE=20000
        idx_to_nodes = data['idx_to_nodes']
        node_to_hierarchy = data['node_to_hierarchy']
        
        self.X_master = np.zeros((SIZE,6),dtype=int)
        for idx, nodes in idx_to_nodes.items():
            if idx<SIZE:
                for node in nodes:
                    self.X_master[idx,node_to_hierarchy[node]] = node
        
        self.X_train = None
        self.y_train = None

        self.state_dict = None
        
    def observe(self, idxs: List[int], activities: np.array, n_iter=50) -> None:
        torch.cuda.empty_cache()
        gc.collect()
        
        # assemble dataset
        if self.X_train is None:
            self.X_train = self.X_master[idxs]
            self.y_train = activities.reshape(-1,1)
        else:
            self.X_train = np.concatenate([self.X_train, self.X_master[idxs]], axis=0)
            self.y_train = np.concatenate([self.y_train, activities.reshape(-1,1)], axis=0)

        X_train = torch.tensor(self.X_train, dtype=torch.int).cuda()
        y_train = torch.tensor(self.y_train, dtype=torch.float16).flatten().cuda()
        
        # train model

        # initialize model
        self.gp_model = GPClassificationModel(X_train)
        torch.cuda.empty_cache()
        gc.collect()
        # if self.state_dict is not None:
        #     self.gp_model.load_state_dict(self.state_dict)
        self.gp_model.cuda()
        self.gp_model.train()
        
        self.likelihood = gpytorch.likelihoods.BernoulliLikelihood()
        self.likelihood.cuda()
        self.likelihood.train()
        
        # optimizer and objective function
        optimizer = torch.optim.Adam(self.gp_model.parameters(), lr=5e-3)
        mll = gpytorch.mlls.VariationalELBO(self.likelihood, self.gp_model, y_train.numel())
        
        for i in range(n_iter):
            optimizer.zero_grad()
            output = self.gp_model(X_train)
            loss = -mll(output, y_train)
            loss.backward()
            if i%10 == 0:
                print('Iter %d/%d - Loss: %.3f' % (i + 1, n_iter, loss.item()), end='\r')
            optimizer.step()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, n_iter, loss.item()))

        del optimizer, mll, loss, X_train, y_train
        torch.cuda.empty_cache()
        gc.collect()

        # self.state_dict = self.gp_model.state_dict()
        
    def sample(self, X) -> np.array:
        torch.cuda.empty_cache()
        gc.collect()
        
        self.gp_model.eval()
        with torch.no_grad():
            f_preds = self.gp_model(X.cuda())
        f_samples = f_preds.sample()
        return f_samples.detach().cpu().numpy()
    
    def predict(self, idxs: List[int]) -> np.array:
        return self.sample(torch.tensor(self.X_master[idxs], dtype=torch.float16))
    
    def select(self, idxs: List[int], n: int) -> List[int]:
        return self.rank(idxs)[:n]

In [12]:
gp_learner = GP_learner(data)
gp_results = iHTS(gp_learner, y, init_idxs)

Iter 50/50 - Loss: 0.651
0 | 0.10: 0.1740
Iter 50/50 - Loss: 0.746
1 | 0.15: 0.2121
Iter 50/50 - Loss: 0.813
2 | 0.20: 0.2480


KeyboardInterrupt: 